This notebook shows the reconstruction of a 3D GRE with two almost identical echotimes sampled with 3D Lissajous variable density shell trajectories.


In [74]:
import sys

sys.path.insert(0, "../src")

import h5py
import torch
import numpy

from juart.conopt.tfs.fourier import nonuniform_transfer_function
from juart.conopt.functional.fourier import nonuniform_fourier_transform_adjoint

# Load preprocessed dataset

In [2]:
data_path = "data/3DLiss_vd_preproc.h5"
with h5py.File(data_path, "r") as f:
    print(f"Dataset holds following data: {f.keys()}")

    print(f"Coilsensitivity info: {f['coilsens'].attrs['info']}")
    print(f"Trajectory info: {f['k'].attrs['info']}")
    print(f"Signal info: {f['d'].attrs['info']}")

    ktraj = f['k'][:]
    coilsens = f['coilsens'][:]
    d = f['d'][:]

    print(f"Coilsensitivity shape {coilsens.shape}")
    print(f"Trajectory shape {ktraj.shape}")
    print(f"Signal shape {d.shape}")
    

Dataset holds following data: <KeysViewHDF5 ['coilsens', 'd', 'k']>
Coilsensitivity info: Shape (Channels, Nx, Ny, Nz).
Trajectory info: Shape (Dimensions, Samples, Echotimes). Scaled in units of cycle/fov
Signal info: Shape (Channels, Samples, Echotimes).
Coilsensitivity shape (8, 156, 156, 156)
Trajectory shape (3, 2001191, 2)
Signal shape (8, 2001191, 2)


# Convert data to JUART format

In [3]:
# Scale trajectory to [-0.5, 0.5]
print(f"Min/Max of trajectory: {ktraj.min()} / {ktraj.max()}")
ktraj = ktraj / (2*ktraj.max())
print(f"Min/Max of trajectory: {ktraj.min()} / {ktraj.max()}")

ktraj = torch.from_numpy(ktraj)
coilsens = torch.from_numpy(coilsens)
d = torch.from_numpy(d)

Min/Max of trajectory: -78.62955474853516 / 78.626708984375
Min/Max of trajectory: -0.5000181198120117 / 0.5


# Perform 3D CG-SENSE reconstruction

In [65]:
AHd = nonuniform_fourier_transform_adjoint(ktraj,d,(156,156,156))
AHd = torch.sum(torch.conj(coilsens[...,None]) * AHd, dim=0)
print(AHd.shape)

torch.Size([156, 156, 156, 2])


In [66]:
H = nonuniform_transfer_function(ktraj,(1,156,156,156,2),oversampling=(2,2,2))
print(H.shape)

torch.Size([1, 312, 312, 312, 2])


In [75]:
cg_solver = SENSE(coilsens,AHd,H)

RuntimeError: The expanded size of the tensor (2) must match the existing size (156) at non-singleton dimension 4.  Target sizes: [8, 156, 156, 156, 2].  Tensor sizes: [8, 156, 156, 156]